# Part 6: Advanced Panoramic Image

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random

In [4]:
im01 = cv2.imread('part1_4/im01.jpg')
im01 = cv2.cvtColor(im01, cv2.COLOR_BGR2RGB)
im02 = cv2.imread('part1_4/im02.jpg')
im02 = cv2.cvtColor(im02, cv2.COLOR_BGR2RGB)
im03 = cv2.imread('part1_4/im03.jpg')
im03 = cv2.cvtColor(im03, cv2.COLOR_BGR2RGB)

In [5]:
# Extract keypoints and descriptors for each image
sift = cv2.SIFT_create()

im01_gray = cv2.cvtColor(im01, cv2.COLOR_BGR2GRAY)
im02_gray = cv2.cvtColor(im02, cv2.COLOR_BGR2GRAY)
im03_gray = cv2.cvtColor(im03, cv2.COLOR_BGR2GRAY)

kp1, des1 = sift.detectAndCompute(im01_gray, None)
kp2, des2 = sift.detectAndCompute(im02_gray, None)
kp3, des3 = sift.detectAndCompute(im03_gray, None)


In [6]:
# find mathches between two images
def match(des1, des2):
    best1 = np.zeros(des1.shape[0])
    best2 = np.zeros(des2.shape[0])

    for i in range(des1.shape[0]):
        distant = np.zeros(des2.shape[0])
        for j in range(des2.shape[0]):
            distant[j] = np.sum((des1[i] - des2[j]) ** 2)
        best1[i] = int(np.argmin(distant))
    
    for i in range(des2.shape[0]):
        distant = np.zeros(des1.shape[0])
        for j in range(des1.shape[0]):
            distant[j] = np.sum((des2[i] - des1[j]) ** 2)       
        best2[i] = int(np.argmin(distant))
    
    best1 = best1.astype(int)
    best2 = best2.astype(int)

    matches = []
    for i in range(best1.shape[0]):
        m = best1[i]
        n = best2[m]
        if i == n:
            matches.append([n, m])
            
    return matches

In [7]:
def homography(points_1, points_2):
    points_1 = np.column_stack((points_1, np.ones(len(points_1))))
    points_2 = np.column_stack((points_2, np.ones(len(points_2))))

    A = np.zeros((2*len(points_1),9))
    for i in range(len(points_1)):
        x1, y1, _ = points_1[i]
        x2, y2, _ = points_2[i]
        A[2*i] = [x1, y1, 1, 0, 0, 0, -x1*x2, -y1*x2, -x2]
        A[2*i + 1] = [0, 0, 0, x1, y1, 1, -x1*y2, -y1*y2, -y2]

    U, D, V_T = np.linalg.svd(A)
    h = V_T[-1]
    H = h.reshape((3,3))
    H = H / H[2,2]
    return H

In [25]:

def RANSAC_homography(kp1, kp2, matches, iteration_times):
    n = 5
    final_H = []
    epsion = 2 # threshold of the distance of inliers
    N = len(matches)
    max_inliers = []
    
    for k in range(iteration_times):
        rdm = random.sample(range(0, N), 5)
        points_1 = []
        points_2 = []
        for j in range(n):
            points_1.append([kp1[int(matches[rdm[j]][0])].pt[0], kp1[int(matches[rdm[j]][0])].pt[1]])
            points_2.append([kp2[int(matches[rdm[j]][1])].pt[0], kp2[int(matches[rdm[j]][1])].pt[1]])

        H = homography(points_1, points_2)
        
        inliers = []
        
        for i in range(N):
            x1 = kp1[int(matches[i][0])].pt[0]
            y1 = kp1[int(matches[i][0])].pt[1]
            x2 = kp2[int(matches[i][1])].pt[0]
            y2 = kp2[int(matches[i][1])].pt[1]
            
            hx1, hy1, hz1 = np.dot(H, [x1, y1, 1])
            hx1 = int(hx1 / hz1)
            hy1 = int(hy1 / hz1)
            distance = np.sqrt((x2 - hx1) ** 2 + (y2 - hy1) ** 2)

            if distance < epsion:
                inliers.append(matches[i])
        
        if len(inliers) > len(max_inliers):
            max_inliers = inliers
    
    points_1 = []
    points_2 = []
    for i in range(len(max_inliers)):
        points_1.append([kp1[int(max_inliers[i][0])].pt[1], kp1[int(max_inliers[i][0])].pt[0]])
        points_2.append([kp2[int(max_inliers[i][1])].pt[1], kp2[int(max_inliers[i][1])].pt[0]])
    final_H = homography(points_1, points_2) 

    return final_H, max_inliers

1. Handling unordered images
   
   use DAG graph to store the images order

My method:
- for all images, find the inliers between two images i and j.  Create a matrix named *map* to store the 'weight'. map[i][j] = len(inliers) / (8.0 + 0.3len(matches)) between i and j
  
 - Create a DAG using *map* matrix. Each image is the node. For each imagei, create a line to the image j which map[i][j] is the biggest, which means for image i , image j is the most similar image.
  
- Then use DFS to find the order that including the most nodes.

In [26]:
import networkx as nx
def dfs_longest_sequence(G, node, current_sequence, visited):
    visited[node] = True
    current_sequence.append(node)
    max_sequence = current_sequence.copy()

    for neighbor in G.successors(node):
        if not visited[neighbor]:
            new_sequence = dfs_longest_sequence(G, neighbor, current_sequence, visited)
            if len(new_sequence) > len(max_sequence):
                max_sequence = new_sequence

    visited[node] = False
    current_sequence.pop()
    return max_sequence

def DAG(map):
    G = nx.DiGraph()

    # 添加结点
    N = map.shape[0]
    for i in range(N):
        G.add_node(i)
    for i in range(N):
        max_weight = -1
        max_index = -1

    # 找到结点 i 到权重最大的结点 j
    for j in range(N):
        if i != j and map[i][j] > max_weight:
            max_weight = map[i][j]
            max_index = j

    if max_index != -1:
        G.add_edge(i, max_index, weight=max_weight)


    visited = [False] * N
    longest_sequence = []
    for i in range(N):
        current_sequence = []
        new_sequence = dfs_longest_sequence(G, i, current_sequence, visited)
        if len(new_sequence) > len(longest_sequence):
            longest_sequence = new_sequence

    print("Longest Directed Sequence:", longest_sequence)
    print("Number of Nodes in Longest Sequence:", len(longest_sequence))

    '''
    pos = nx.spring_layout(G)  # 定义结点位置
    labels = {i: f"image {i}" for i in G.nodes()}
    nx.draw(G, pos, with_labels=True, labels=labels, node_size=500, node_color="skyblue", font_size=12, font_color="black", arrows=True)
    edge_labels = {(i, j): map[i][j] for i, j in G.edges()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    plt.title("Directed Graph")
    plt.show()
    '''
    
    return longest_sequence

In [27]:
#When use RANSAC between two images, add a threshold to judge if the two image need to be stitched.
# That is if the len(inliers) > 8.0 + 0.3*len(matches)
def stitch_order(all_kps, all_des, iteration_times):
    '''
    all_kps: a list contain all keypoints of each image. all_kps = [kp1, kp2, kp3, ...]
    all_des: a list contain all descriptors of each image. all_des = [des1, des2, des3, ...]
    iteration_times: the iteration times of RANSAC 
    '''
    N = len(all_kps) # the number of unordered images
    map = np.zeros((N, N)) # use matrix to store the images stitch weight

    # for each image, find the best match image
    for i in range(N):

        kp_1 = all_kps[i]
        des_1 = all_des[i]
        print(i)
        for j in range(N):
            if j == i:
                map[i][j] = 0  # the same image do not need stitch
                continue
            else:
                print(j)
                kp_2 = all_kps[j]
                des_2 = all_des[j]
                matches = match(des_1, des_2)
                H, inliers = RANSAC_homography(kp_1, kp_2, matches, iteration_times)

                #if len(inliers) > 8.0 + 0.3*len(matches):
                map[i][j] = len(inliers) / (8.0 + 0.3*len(matches))
                #print(f'image{i} and image{j} need stitch')
                #else:
                    #map[i][j] = 0
    
    order = DAG(map)
    return order


In [28]:
all_kps = [kp1, kp2, kp3]
all_des = [des1, des2, des3]
order = stitch_order(all_kps, all_des, 10000)
order

0
1
2
1
0
2
2
0
1
Longest Directed Sequence: [2, 1]
Number of Nodes in Longest Sequence: 2


[2, 1]

Then just use the order to stitch the corresponding image using the method in part5.